# 1. Data & Model Loading

This notebook prepares the data and models used for the subsequent optimisation pipeline. This is to emulate a non-compressed model training and evaluation process, where the model is adapted to a specific dataset and then exported for further compression for embedded deployment.

The process is defined as such:
* A Torch dataset (already split into train and val) and model are loaded. Those must be specialized for classification tasks, but are agnostic
of the modality.
* The model"s classification head is adapted to the number of classes in the dataset, trained on the training set while freezing the backbone, and evaluated on the validation set.
* The whole model (backbone + classification head) is then adapted to the dataset by freezing all layers except the classification head, which is trained on the training set.
* The adapted model is then exported as a Torch model for later use in the optimisation pipeline.

An image MobileNetV2 model with a classification head adapted to the CIFAR-10 dataset is used as an example in this notebook.

## Setup

In [1]:
import torch

from nnopt.model.train import adapt_model_head_to_dataset
from nnopt.model.eval import eval_model
from nnopt.model.const import DEVICE, DTYPE, AMP_ENABLE
from nnopt.recipes.mobilenetv2_cifar10 import init_mobilenetv2_cifar10_model, get_cifar10_datasets, save_mobilenetv2_cifar10_model, load_mobilenetv2_cifar10_model

/home/pbeuran/repos/nnopt/.venv/lib/python3.12/site-packages/openvino/runtime/__init__.py:10: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  warnings.warn(
2025-06-13 19:34:56,808 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Using device: cuda, dtype: torch.bfloat16


# MobileNetV2 and CIFAR-10 adaptation

In [2]:
mobilenetv2 = init_mobilenetv2_cifar10_model()
cifar10_train_dataset, cifar10_val_dataset, cifar10_test_dataset = get_cifar10_datasets()

# Adapt the MobileNetV2 model to CIFAR-10 dataset
mobilenetv2_cifar10_baseline = adapt_model_head_to_dataset(
    model=mobilenetv2,
    train_dataset=cifar10_train_dataset,
    val_dataset=cifar10_val_dataset,
    batch_size=64,  # Adjust batch size as needed
    head_train_epochs=5,  # Train head for fewer epochs
    fine_tune_epochs=5,  # Fine-tune for fewer epochs
    optimizer_cls=torch.optim.Adam,  # Use Adam optimizer
    head_train_lr=0.001,  # Learning rate for head training
    fine_tune_lr=0.0001,  # Learning rate for fine-tuning
    use_amp=AMP_ENABLE,  # Use mixed precision training for efficiency
    device=DEVICE, # Should be CUDA is available or CPU
    dtype=DTYPE # Should be torch.float32 or torch.float16
)

2025-06-13 19:34:56,816 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Loading MobileNetV2 model with weights: MobileNet_V2_Weights.IMAGENET1K_V1, to_quantize: False, is_quantized: False
2025-06-13 19:34:56,896 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Replacing head of the model to match 10 classes
2025-06-13 19:34:56,898 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Loading existing training and validation datasets...
2025-06-13 19:34:58,540 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Loading existing test dataset...
2025-06-13 19:34:58,870 - nnopt.model.train - INFO - Training head of the model with backbone frozen...
Epoch 1/5 [Validation]: 100%|██████████| 79/79 [00:02<00:00, 38.82it/s, acc=0.6574, cpu=3.1%, gpu_mem=15.8/24.0GB (65.9%), gpu_util=43.0%, loss=1.0329, ram=10.6/30.9GB (45.2%), samples/s=1087.9] 


Epoch 1/5, Train Loss: 1.4514, Train Acc: 0.4921, Train Throughput: 4289.79 samples/s | Val Loss: 0.9768, Val Acc: 0.6574, Val Throughput: 7879.72 samples/s | CPU Usage: 11.50% | RAM Usage: 10.5/30.9GB (44.6%) | GPU 0 Util: 43.00% | GPU 0 Mem: 15.8/24.0GB (65.9%)


Epoch 2/5 [Validation]: 100%|██████████| 79/79 [00:02<00:00, 37.44it/s, acc=0.6588, cpu=3.3%, gpu_mem=15.8/24.0GB (65.9%), gpu_util=41.0%, loss=1.0219, ram=10.6/30.9GB (45.1%), samples/s=1260.4] 


Epoch 2/5, Train Loss: 1.3412, Train Acc: 0.5272, Train Throughput: 4523.97 samples/s | Val Loss: 0.9767, Val Acc: 0.6588, Val Throughput: 7913.98 samples/s | CPU Usage: 12.30% | RAM Usage: 10.4/30.9GB (44.3%) | GPU 0 Util: 41.00% | GPU 0 Mem: 15.8/24.0GB (65.9%)


Epoch 3/5 [Validation]: 100%|██████████| 79/79 [00:02<00:00, 38.05it/s, acc=0.6438, cpu=3.4%, gpu_mem=15.8/24.0GB (65.9%), gpu_util=42.0%, loss=0.8809, ram=10.6/30.9GB (44.9%), samples/s=1326.4] 


...

Epoch 3/5, Train Loss: 0.6211, Train Acc: 0.7834, Train Throughput: 1757.00 samples/s | Val Loss: 0.2856, Val Acc: 0.9028, Val Throughput: 8108.88 samples/s | CPU Usage: 11.90% | RAM Usage: 10.5/30.9GB (44.6%) | GPU 0 Util: 45.00% | GPU 0 Mem: 18.3/24.0GB (76.3%)


Epoch 4/5 [Validation]: 100%|██████████| 79/79 [00:02<00:00, 37.23it/s, acc=0.9202, cpu=0.0%, gpu_mem=18.3/24.0GB (76.3%), gpu_util=46.0%, loss=0.6335, ram=10.7/30.9GB (45.5%), samples/s=1301.9] 


Epoch 4/5, Train Loss: 0.5777, Train Acc: 0.8005, Train Throughput: 1760.24 samples/s | Val Loss: 0.2292, Val Acc: 0.9202, Val Throughput: 7791.42 samples/s | CPU Usage: 12.30% | RAM Usage: 10.5/30.9GB (44.6%) | GPU 0 Util: 46.00% | GPU 0 Mem: 18.3/24.0GB (76.3%)


Epoch 5/5 [Validation]: 100%|██████████| 79/79 [00:02<00:00, 37.42it/s, acc=0.9242, cpu=0.0%, gpu_mem=18.3/24.0GB (76.3%), gpu_util=40.0%, loss=1.0600, ram=10.7/30.9GB (45.5%), samples/s=1277.9] 

Epoch 5/5, Train Loss: 0.5493, Train Acc: 0.8087, Train Throughput: 1767.28 samples/s | Val Loss: 0.2148, Val Acc: 0.9242, Val Throughput: 8054.70 samples/s | CPU Usage: 11.50% | RAM Usage: 10.5/30.9GB (44.7%) | GPU 0 Util: 40.00% | GPU 0 Mem: 18.3/24.0GB (76.3%)


In [3]:
# Evaluate the adapted model on the validation and test set
val_metrics = eval_model(
    model=mobilenetv2_cifar10_baseline,
    test_dataset=cifar10_val_dataset,
    batch_size=64,  # Adjust batch size as needed
    device=DEVICE,
    use_amp=AMP_ENABLE,
    dtype=DTYPE
)

test_metrics = eval_model(
    model=mobilenetv2_cifar10_baseline,
    test_dataset=cifar10_test_dataset,
    batch_size=64,  # Adjust batch size as needed
    device=DEVICE,
    use_amp=AMP_ENABLE,
    dtype=DTYPE
)
print(f"Validation accuracy of the adapted MobileNetV2 on CIFAR-10: {val_metrics['accuracy']:.2f}")
print(f"Test accuracy of the adapted MobileNetV2 on CIFAR-10: {test_metrics['accuracy']:.2f}")

2025-06-13 19:41:44,351 - nnopt.model.eval - INFO - Starting evaluation on device: cuda, dtype: torch.bfloat16, batch size: 64
2025-06-13 19:41:44,354 - nnopt.model.eval - INFO - Starting warmup for 5 batches...
[Warmup]: 100%|██████████| 5/5 [00:00<00:00, 12.49it/s]
2025-06-13 19:41:44,852 - nnopt.model.eval - INFO - Warmup complete.
[Evaluation]: 100%|██████████| 79/79 [00:02<00:00, 37.33it/s, acc=0.9242, cpu=9.4%, gpu_mem=18.3/24.0GB (76.3%), gpu_util=45.0%, loss=1.0600, ram=10.8/30.9GB (45.5%), samples/s=1221.4] 
2025-06-13 19:41:46,974 - nnopt.model.eval - INFO - Starting evaluation on device: cuda, dtype: torch.bfloat16, batch size: 64
2025-06-13 19:41:46,977 - nnopt.model.eval - INFO - Starting warmup for 5 batches...


Evaluation Complete: Avg Loss: 0.2148, Accuracy: 0.9242
Throughput: 7911.22 samples/sec | Avg Batch Time: 8.00 ms | Avg Sample Time: 0.13 ms
System Stats: CPU Usage: 14.50% | RAM Usage: 10.5/30.9GB (44.7%) | GPU 0 Util: 45.00% | GPU 0 Mem: 18.3/24.0GB (76.3%)


[Warmup]: 100%|██████████| 5/5 [00:00<00:00, 14.79it/s]
2025-06-13 19:41:47,412 - nnopt.model.eval - INFO - Warmup complete.
[Evaluation]: 100%|██████████| 157/157 [00:04<00:00, 38.45it/s, acc=0.9273, cpu=3.2%, gpu_mem=18.3/24.0GB (76.2%), gpu_util=42.0%, loss=0.1041, ram=10.7/30.9GB (45.6%), samples/s=614.1]  

Evaluation Complete: Avg Loss: 0.2088, Accuracy: 0.9273
Throughput: 7854.46 samples/sec | Avg Batch Time: 8.11 ms | Avg Sample Time: 0.13 ms
System Stats: CPU Usage: 11.00% | RAM Usage: 10.5/30.9GB (44.7%) | GPU 0 Util: 42.00% | GPU 0 Mem: 18.3/24.0GB (76.2%)
Validation accuracy of the adapted MobileNetV2 on CIFAR-10: 0.92
Test accuracy of the adapted MobileNetV2 on CIFAR-10: 0.93


In [4]:
# Export the adapted model
save_mobilenetv2_cifar10_model(
    model=mobilenetv2_cifar10_baseline,
    metrics_values={
        "val_metrics": val_metrics,
        "test_metrics": test_metrics,
    },
    version="mobilenetv2_cifar10/fp32/baseline",
)

2025-06-13 19:41:51,561 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Model saved to /home/pbeuran/repos/nnopt/models/mobilenetv2_cifar10/fp32/baseline/model.pt
2025-06-13 19:41:51,601 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Model state_dict saved to /home/pbeuran/repos/nnopt/models/mobilenetv2_cifar10/fp32/baseline/state_dict.pt
2025-06-13 19:41:51,602 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Metadata saved to /home/pbeuran/repos/nnopt/models/mobilenetv2_cifar10/fp32/baseline/metadata.json
2025-06-13 19:41:51,603 - nnopt.model.prune - INFO - Making pruning permanent by removing reparameterization...
2025-06-13 19:41:51,603 - nnopt.model.prune - WARNING - No pruning reparameterization was removed. Was the model pruned?
2025-06-13 19:41:51,603 - nnopt.recipes.mobilenetv2_cifar10 - INFO - Saving model in JIT script format...
2025-06-13 19:41:51,929 - nnopt.recipes.mobilenetv2_cifar10 - INFO - JIT script model saved to /home/pbeuran/repos/nnopt/models/mobilenetv2_cifar10/fp32/base

# Analysis

## GPU FP32

In [5]:
# Evaluate the adapted model on the validation and test set on GPU
val_metrics = eval_model(
    model=mobilenetv2_cifar10_baseline,
    test_dataset=cifar10_val_dataset,
    batch_size=64,  # Adjust batch size as needed
    device="cuda",
    use_amp=False,
    dtype=torch.float32
)

test_metrics = eval_model(
    model=mobilenetv2_cifar10_baseline,
    test_dataset=cifar10_test_dataset,
    batch_size=64,  # Adjust batch size as needed
    device="cuda",
    use_amp=False,
    dtype=torch.float32
)

2025-06-13 19:41:52,621 - nnopt.model.eval - INFO - Starting evaluation on device: cuda, dtype: torch.float32, batch size: 64
2025-06-13 19:41:52,646 - nnopt.model.eval - INFO - Starting warmup for 5 batches...
[Warmup]: 100%|██████████| 5/5 [00:00<00:00,  9.73it/s]
2025-06-13 19:41:53,258 - nnopt.model.eval - INFO - Warmup complete.
[Evaluation]: 100%|██████████| 79/79 [00:02<00:00, 37.77it/s, acc=0.9244, cpu=7.2%, gpu_mem=18.9/24.0GB (78.7%), gpu_util=61.0%, loss=1.0547, ram=10.9/30.9GB (46.0%), samples/s=432.3]  
2025-06-13 19:41:55,355 - nnopt.model.eval - INFO - Starting evaluation on device: cuda, dtype: torch.float32, batch size: 64
2025-06-13 19:41:55,358 - nnopt.model.eval - INFO - Starting warmup for 5 batches...


Evaluation Complete: Avg Loss: 0.2183, Accuracy: 0.9244
Throughput: 5278.00 samples/sec | Avg Batch Time: 11.99 ms | Avg Sample Time: 0.19 ms
System Stats: CPU Usage: 15.30% | RAM Usage: 10.7/30.9GB (45.4%) | GPU 0 Util: 61.00% | GPU 0 Mem: 18.9/24.0GB (78.7%)


[Warmup]: 100%|██████████| 5/5 [00:00<00:00, 12.47it/s]
2025-06-13 19:41:55,871 - nnopt.model.eval - INFO - Warmup complete.
[Evaluation]: 100%|██████████| 157/157 [00:03<00:00, 40.25it/s, acc=0.9267, cpu=1.6%, gpu_mem=18.8/24.0GB (78.5%), gpu_util=62.0%, loss=0.0984, ram=10.9/30.9GB (46.0%), samples/s=958.4]  

Evaluation Complete: Avg Loss: 0.2120, Accuracy: 0.9267
Throughput: 5288.75 samples/sec | Avg Batch Time: 12.04 ms | Avg Sample Time: 0.19 ms
System Stats: CPU Usage: 11.40% | RAM Usage: 10.7/30.9GB (45.4%) | GPU 0 Util: 62.00% | GPU 0 Mem: 18.8/24.0GB (78.5%)


In [6]:
# Print the val metrics
import yaml
print("- Validation Metrics:")
yaml_str = yaml.dump(val_metrics, default_flow_style=False)
print(yaml_str)

# Print the test metrics
print("- Test Metrics:")
yaml_str = yaml.dump(test_metrics, default_flow_style=False)
print(yaml_str)

- Validation Metrics:
accuracy: 0.9244
avg_loss: 0.21826240797042848
avg_time_per_batch: 11.991510962086723
avg_time_per_sample: 0.18946587320097025
params_stats:
  bn_param_params: 34112
  float_bias_params: 10
  float_weight_params: 2202560
  int_weight_params: 0
  other_float_params: 0
  total_params: 2236682
samples_per_second: 5277.995361936659

- Test Metrics:
accuracy: 0.9267
avg_loss: 0.21195748742818832
avg_time_per_batch: 12.043359070008059
avg_time_per_sample: 0.1890807373991265
params_stats:
  bn_param_params: 34112
  float_bias_params: 10
  float_weight_params: 2202560
  int_weight_params: 0
  other_float_params: 0
  total_params: 2236682
samples_per_second: 5288.746033865529



## CPU FP32

In [7]:
# Evaluate the adapted model on the validation and test set on CPU
val_metrics = eval_model(
    model=mobilenetv2_cifar10_baseline,
    test_dataset=cifar10_val_dataset,
    batch_size=32,  # Adjust batch size as needed
    device="cpu",
    use_amp=False,
    dtype=torch.float32
)

test_metrics = eval_model(
    model=mobilenetv2_cifar10_baseline,
    test_dataset=cifar10_test_dataset,
    batch_size=32,  # Adjust batch size as needed
    device="cpu",
    use_amp=False,
    dtype=torch.float32
)

2025-06-13 19:41:59,818 - nnopt.model.eval - INFO - Starting evaluation on device: cpu, dtype: torch.float32, batch size: 32
2025-06-13 19:41:59,857 - nnopt.model.eval - INFO - Starting warmup for 5 batches...
[Warmup]: 100%|██████████| 5/5 [00:02<00:00,  2.00it/s]
2025-06-13 19:42:02,472 - nnopt.model.eval - INFO - Warmup complete.
[Evaluation]: 100%|██████████| 157/157 [01:07<00:00,  2.32it/s, acc=0.9242, cpu=43.6%, loss=1.0552, ram=11.0/30.9GB (47.6%), samples/s=138.8]
2025-06-13 19:43:10,132 - nnopt.model.eval - INFO - Starting evaluation on device: cpu, dtype: torch.float32, batch size: 32
2025-06-13 19:43:10,136 - nnopt.model.eval - INFO - Starting warmup for 5 batches...


Evaluation Complete: Avg Loss: 0.2183, Accuracy: 0.9242
Throughput: 74.58 samples/sec | Avg Batch Time: 426.99 ms | Avg Sample Time: 13.41 ms
System Stats: CPU Usage: 14.20% | RAM Usage: 10.8/30.9GB (46.8%)


[Warmup]: 100%|██████████| 5/5 [00:02<00:00,  2.01it/s]
2025-06-13 19:43:12,743 - nnopt.model.eval - INFO - Warmup complete.
[Evaluation]: 100%|██████████| 313/313 [02:28<00:00,  2.11it/s, acc=0.9266, cpu=33.4%, loss=0.0986, ram=11.0/30.9GB (47.4%), samples/s=81.8]

Evaluation Complete: Avg Loss: 0.2120, Accuracy: 0.9266
Throughput: 67.69 samples/sec | Avg Batch Time: 471.98 ms | Avg Sample Time: 14.77 ms
System Stats: CPU Usage: 12.80% | RAM Usage: 10.8/30.9GB (46.7%)


In [8]:
# Print the val metrics
import yaml
print("- Validation Metrics:")
yaml_str = yaml.dump(val_metrics, default_flow_style=False)
print(yaml_str)

# Print the test metrics
print("- Test Metrics:")
yaml_str = yaml.dump(test_metrics, default_flow_style=False)
print(yaml_str)

- Validation Metrics:
accuracy: 0.9242
avg_loss: 0.21828886901140213
avg_time_per_batch: 426.994085019183
avg_time_per_sample: 13.407614269602345
params_stats:
  bn_param_params: 34112
  float_bias_params: 10
  float_weight_params: 2202560
  int_weight_params: 0
  other_float_params: 0
  total_params: 2236682
samples_per_second: 74.58448459896354

- Test Metrics:
accuracy: 0.9266
avg_loss: 0.2119856424972415
avg_time_per_batch: 471.9812060444592
avg_time_per_sample: 14.773011749191573
params_stats:
  bn_param_params: 34112
  float_bias_params: 10
  float_weight_params: 2202560
  int_weight_params: 0
  other_float_params: 0
  total_params: 2236682
samples_per_second: 67.69100417555163



## Conclusions

* Accuracy is ~92.8% for CIFAR-10 with MobileNetV2, with fast convergence for so few epochs.
* GPU is ~75 time faster than CPU for both training and evaluation, which is to be expected considering architecture differences.
* Thus, if wanting to run the model on a CPU for embedded cases, and expect high throughput during inference with little-to-no accuracy loss, the model should be optimised for the CPU. This can be done with pruning, quantization, knowledge distillation.
* Pruning and quantization are good candidates and explored in the next notebooks, while knowledge distillation isn't because of the already efficient architecture of MobileNetV2.